In [1]:
import os
import sys
sys.path.append("..")
import phage_init

In [2]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations


In [3]:
#from numba import jit,vectorize

In [4]:
all_fasta=(os.path.join(phage_init.fasta_dir,"major_capsid.fasta"),os.path.join(phage_init.fasta_dir,"minor_capsid.fasta"),
           os.path.join(phage_init.fasta_dir,"baseplate.fasta"),os.path.join(phage_init.fasta_dir,"major_tail.fasta"),
           os.path.join(phage_init.fasta_dir,"minor_tail.fasta"),os.path.join(phage_init.fasta_dir,"portal.fasta"),
           os.path.join(phage_init.fasta_dir,"tail_fiber.fasta"),os.path.join(phage_init.fasta_dir,"tail_sheath.fasta"),
           os.path.join(phage_init.fasta_dir,"collar.fasta"),os.path.join(phage_init.fasta_dir,"htj.fasta"))

In [ ]:

def extract_all(fasta_list):
    AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
    SC=["1","2","3","4","5","6","7"]
    di_pep = [''.join(i) for i in itertools.product(AA, repeat = 2)]
    tri_pep = [''.join(i) for i in itertools.product(AA, repeat = 3)]
    di_sc = [''.join(i) for i in itertools.product(SC, repeat = 2)]
    tri_sc = [''.join(i) for i in itertools.product(SC, repeat = 3)]
    prot_class=0;
    myseq="AILMVNQSTGPCHKPDEFWY"
    trantab2=myseq.maketrans("AILMVNQSTGPCHKPDEFWY","11111222233455566777")
    arr = numpy.empty((0,8800), dtype=numpy.float)
    class_arr = numpy.empty((0), dtype=numpy.int)
    for file in fasta_list:
        print('####################' + file)
        for record in SeqIO.parse(file, "fasta"):
            ll=len(record.seq)
            seqq=record.seq.__str__().upper()
            seqqq=seqq.replace('X','A').replace('J','L').replace('*','A')
            X = ProteinAnalysis(seqqq)
            tt= [X.isoelectric_point(), X.instability_index(),ll,X.aromaticity(),
                X.molar_extinction_coefficient()[0],X.molar_extinction_coefficient()[1],
                X.gravy(),X.molecular_weight()]
            tt_n = numpy.asarray(tt,dtype=numpy.float)
            myseq=seqq.translate(trantab2)
            
            di_pep_count=[seqq.count(i)/(ll-1) for i in di_pep]
            di_pep_count_n = numpy.asarray(di_pep_count,dtype=numpy.float)
            
            tri_pep_count=[seqq.count(i)/(ll-2) for i in tri_pep]
            tri_pep_count_n = numpy.asarray(tri_pep_count,dtype=numpy.float)
            
            di_sc_count=[myseq.count(i)/(ll-1) for i in di_sc]
            di_sc_count_n = numpy.asarray(di_sc_count,dtype=numpy.float)
    
            tri_sc_count=[myseq.count(i)/(ll-2) for i in tri_sc]
            tri_sc_count_n = numpy.asarray(tri_sc_count,dtype=numpy.float)
    
            cat_n= numpy.concatenate((di_pep_count_n,tri_pep_count_n,di_sc_count_n,tri_sc_count_n,tt_n))
            #print(cat_n.shape)
            cat_n = cat_n.reshape((1,cat_n.shape[0]))

            arr = numpy.append(arr,cat_n , axis=0)
            class_arr = numpy.append(class_arr,prot_class)
        prot_class+=1
    return (arr,class_arr)
    

In [ ]:
(arr,class_arr)=extract_all(all_fasta)

####################/home/adrian/git-presentations/keras/phage_protein/fasta/major_capsid.fasta


In [9]:
print(arr.shape)
arr_z=numpy.apply_along_axis(stats.zscore,0,arr)



(25675, 8800)


In [11]:
nb_classes = 10
one_hot_targets = numpy.eye(nb_classes)[class_arr]

In [12]:
final = numpy.concatenate((arr_z, one_hot_targets), axis=1)

In [15]:
mean_arr=numpy.apply_along_axis(numpy.mean,0,arr)
#print(mean_arr)
print(mean_arr.shape)
std_arr=numpy.apply_along_axis(numpy.std,0,arr)
print(std_arr.shape)

(8800,)
(8800,)


In [16]:
import pickle
##number of proteins for training
tt=25000
##number of features
f_num=arr.shape[1]
pickle.dump(final[0:tt,0:f_num], open( "train_x.p", "wb" ) )
pickle.dump(final[tt:,0:f_num],  open( "test_x.p" , "wb" ) )
pickle.dump(final[0:tt,f_num:], open( "train_y.p", "wb" ) )
pickle.dump(final[tt:,f_num:],  open( "test_y.p" , "wb" ) )
pickle.dump(mean_arr,  open( "mean.p" , "wb" ) )
pickle.dump(std_arr,  open( "std.p" , "wb" ) )